<a href="https://colab.research.google.com/github/KeHang-Zhu/panic_spread/blob/main/4_20_CA_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import requests
import regex as re

In [14]:
import os
os.environ['NotebookApp.iopub_data_rate_limit'] = str(1000000000)

In [15]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")
print(sentiment_analysis("I love this!"))

[{'label': 'POSITIVE', 'score': 0.9988656044006348}]


Want to filter the CVS and only keep the entry/rows that have the year in First Issue Date < 1940 and the year in Last Issue Date > 1920

In [17]:
!pip install symspellpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
from symspellpy import SymSpell
import pkg_resources

In [19]:
symspell = SymSpell()
dict_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt") # Get file path for dictionary
symspell.load_dictionary(dict_path, 0, 1) # Load in dictionary
eng_dictionary = symspell.words # Get keyed list of english words

In [20]:
from bs4 import BeautifulSoup

In [21]:
# read in the CSV file
df = pd.read_csv('/content/4_20_CA_newspapers - 4_20_CA_newspapers.csv', encoding='latin-1')

print(df.columns)
df.columns = [c.strip() for c in df.columns]

# convert the 'First Issue Date' and 'Last Issue Date' columns to datetime objects
df['First Issue Date'] = pd.to_datetime(df['First Issue Date'])
df['Last Issue Date'] = pd.to_datetime(df['Last Issue Date'])

# extract the year from the 'First Issue Date' and 'Last Issue Date' columns
df['First Issue Year'] = df['First Issue Date'].dt.year
df['Last Issue Year'] = df['Last Issue Date'].dt.year

# filter the dataframe to only keep entries with a year greater than 1920
df_filtered = df[(df['First Issue Year'] < 1940) & (df['Last Issue Year'] > 1920)]

# print out the filtered dataframe
print(df_filtered)

Index(['Persistent Link', 'State', 'Title', 'LCCN', 'OCLC', 'ISSN',
       'No. of Issues', 'First Issue Date', 'Last Issue Date', 'More Info'],
      dtype='object')
                                        Persistent Link    State  \
2     https://chroniclingamerica.loc.gov/lccn/sn8503...  Alabama   
22    https://chroniclingamerica.loc.gov/lccn/202121...   Alaska   
26    https://chroniclingamerica.loc.gov/lccn/sn8402...   Alaska   
27    https://chroniclingamerica.loc.gov/lccn/sn9304...   Alaska   
34    https://chroniclingamerica.loc.gov/lccn/sn9405...   Alaska   
...                                                 ...      ...   
3992  https://chroniclingamerica.loc.gov/lccn/sn9206...  Wyoming   
3999  https://chroniclingamerica.loc.gov/lccn/sn9206...  Wyoming   
4000  https://chroniclingamerica.loc.gov/lccn/sn9206...  Wyoming   
4001  https://chroniclingamerica.loc.gov/lccn/sn9206...  Wyoming   
4004  https://chroniclingamerica.loc.gov/lccn/sn8300...  Wyoming   

                

Loop through all the sn# & years

In [51]:
def get_url(lccn, year):
    # Example url:
    url = f'https://chroniclingamerica.loc.gov/lccn/{lccn}/issues/{year}'
    # url = 'https://chroniclingamerica.loc.gov/lccn/sn86072192/issues/1897/'
    lccn_label = lccn.strip() 

    session = requests.Session()
    response = session.get(url).text
    # print(response)

    # Parse out issue dates from the response using regex
    # pattern = rf'<a href="/lccn/{lccn}/{year}.*/\">'
    # pattern = rf'<a href="/lccn/{lccn}/{year}-\d{2}-\d{2}/*/">'

    pattern = r'<a href="/lccn/'+ lccn.strip() + '/' + str(year) +'-.*?>'
    matches = re.findall(pattern, response)
    
    # Return a dictionary indexed by lccn and year
    return {'lccn': lccn, 'year': year, 'matches': matches}

In [52]:
import os

def save_matches_to_file(lccn, year, matches):
  # Helper function to save matches to file
  directory = f'/content/urls/{lccn.strip()}/'
  print(f'{directory}{year}.txt')
  if not os.path.exists(directory):
    os.makedirs(directory)
  with open(f'{directory}/{year}.txt', 'w') as outfile:
    outfile.writelines(matches)

def scrape_urls(df_filtered):
  # Main function to scrape URLs and save matches to file
  # for index, row in df_filtered.iterrows():
  #   print(row)
  for index, row in df_filtered.iterrows():
    lccn = row['LCCN']
    # first_year = row['First Issue Year']
    # last_year = row['Last Issue Year']
    first_year = int(row['First Issue Year'])
    last_year = int(row['Last Issue Year'])
    for year in range(first_year, last_year+1):
      # result = get_url(lccn, year)
      # matches = result['matches'
      if int(year) >= 1920 and int(year) <= 1940:
        print("LCCN",lccn,"year:",year)
        texts_score = get_text(lccn, year)
        print("LCCN",lccn,"year:",year, 'score:',texts_score)
        # save_matches_to_file(lccn, year, matches)
        save_matches_to_file(lccn, year, texts_score)
      else:
        continue

In [53]:
import re

# html_text = '<tr><td class="single sun"><a href="/lccn/sn86072192/1897-08-22/ed-1/">22</a></td><td class="noissues mon">23</td><td class="single tue"><a href="/lccn/sn86072192/1897-08-24/ed-1/">24</a></td><td class="single wed"><a href="/lccn/sn86072192/1897-08-25/ed-1/">25</a></td><td class="single thu"><a href="/lccn/sn86072192/1897-08-26/ed-1/">26</a></td><td class="single fri"><a href="/lccn/sn86072192/1897-08-27/ed-1/">27</a></td><td class="single sat"><a href="/lccn/sn86072192/1897-08-28/ed-1/">28</a></td></tr>'
html_text =  ' <a href="/lccn/sn86072192/1897-08-01/ed-1/seq-1/">'
# Use regular expression to find all matches
# pattern = r'<a href="/lccn/sn86072192/1897-.*?>'
lccn = ' sn86072192 '
year = 1897
pattern = r'<a href="/lccn/'+ lccn.strip() + '/' + str(year) +'-\d{2}-\d{2}/ed-1/seq-\d+/">'
matches = re.findall(pattern, html_text)

# Print all matches
for match in matches:
    print(match)

<a href="/lccn/sn86072192/1897-08-01/ed-1/seq-1/">


In [54]:
def get_word_rate(text, dictionary): # Function to get % of words in text that appear in a dictionary
  num_words = 0
  for word in text.split():
    if word.lower() in dictionary.keys():
      num_words += 1

  return num_words / len(text.split())

In [55]:
def chunk_text(text, chunk_size):
  chunks = []
  words = text.split()
  i = 0
  while i + chunk_size < len(words):
    chunks.append(' '.join(words[i: i + chunk_size]))
    i += chunk_size
  chunks.append(' '.join(words[i:]))
  return chunks

In [56]:
chunk_size = 200
WORD_RATE_THRESHOLD = .6 # Set this to whateve

In [57]:
def get_text(lccn, year):
    # Fetch the URLs of all issues in the given year
    url_data = get_url(lccn, year)
    issue_urls = url_data['matches']
    head = 'https://chroniclingamerica.loc.gov/'
    tail = 'ocr/'
    text = ''
    score = 0
    chunk_number = 0
    old_url = head
    # Loop through all issue URLs and extract the text
    for issue_url in issue_urls:
        # Extract the URL from the issues_url variable
        url = issue_url.strip('<a href="').strip('">')

        # Combine the URL with the head variable to form the full URL
        full_url = head + url
        response = requests.get(full_url)
        html_content = response.text
        print(full_url)
        # Use regular expression to find all matches
        # pattern = r'<a href="/lccn/'+ lccn.strip() + '/' + str(year) +'-.*?/ed-1/seq-\d+/>'
        pattern = r'<a href="/lccn/'+ lccn.strip() + '/' + str(year) +'-\d{2}-\d{2}/ed-1/seq-\d+/">'
        # r'<a href="/lccn/sn86072192/1897-08-22/ed-1/seq-\d+/'
        text_matches = re.findall(pattern, html_content)

        for text_url in text_matches:
            t_url = text_url.strip('<a href="').strip('">')
            ocr_url = head+ t_url + tail
            print(ocr_url)
            if (ocr_url == old_url):
                continue
            else:
        # Fetch the HTML content of the page with "ocr" in the URL
        # ocr_url = issue_url.replace('/ed-', '/ocr/ed-')
                response = requests.get(ocr_url)
                ocr_content = response.text

                # Parse the HTML content using BeautifulSoup
                soup = BeautifulSoup(ocr_content, 'html.parser')

                # Extract the text content of the desired HTML element(s)
                
                for elem in soup.find_all('div'):
                    if elem.find('p') is not None:
                        # text += elem.find('p').get_text() + '\n'
                        text = elem.find('p').get_text()
                        # print(len(text))
                        chunks = chunk_text(text, chunk_size)
                        for chunk in chunks:
                          rate = get_word_rate(chunk, eng_dictionary)
                          #  print(rate)
                          if (rate > WORD_RATE_THRESHOLD):
                              # print('I am here', chunk)
                              # data =sentiment_analysis(chunk)
                              # # print(data)
                              chunk_number += 1
                              # if data[0]['label'] == 'NEGATIVE':
                              #     score += -1 * data[0]['score']
                              # elif data[0]['label'] == 'POSITIVE':
                              #     score += data[0]['score']
                        print('current score', score, 'current chunk number', chunk_number)
                        # print('ocr url:', ocr_url)
                        old_url = ocr_url
                        text = ''

            

                    # print(elem.find('p').get_text())
            # Print the extracted text
           
    return str(score), str(chunk_number)


In [58]:
scrape_urls(df_filtered)

LCCN sn85038485 year: 1920
https://chroniclingamerica.loc.gov//lccn/sn85038485/1920-01-01/ed-1/
https://chroniclingamerica.loc.gov//lccn/sn85038485/1920-01-01/ed-1/seq-1/ocr/
current score 0 current chunk number 15
https://chroniclingamerica.loc.gov//lccn/sn85038485/1920-01-01/ed-1/seq-1/ocr/
https://chroniclingamerica.loc.gov//lccn/sn85038485/1920-01-01/ed-1/seq-2/ocr/
current score 0 current chunk number 32
https://chroniclingamerica.loc.gov//lccn/sn85038485/1920-01-01/ed-1/seq-2/ocr/
https://chroniclingamerica.loc.gov//lccn/sn85038485/1920-01-01/ed-1/seq-3/ocr/
current score 0 current chunk number 48
https://chroniclingamerica.loc.gov//lccn/sn85038485/1920-01-01/ed-1/seq-3/ocr/
https://chroniclingamerica.loc.gov//lccn/sn85038485/1920-01-01/ed-1/seq-4/ocr/
current score 0 current chunk number 75
https://chroniclingamerica.loc.gov//lccn/sn85038485/1920-01-01/ed-1/seq-4/ocr/
https://chroniclingamerica.loc.gov//lccn/sn85038485/1920-01-01/ed-1/seq-5/ocr/
current score 0 current chunk num

KeyboardInterrupt: ignored

In [ ]:
xxx = "I am here OESEEi THE AGE-HERALD.. J1 , _ . ..., __'__ _ _ _ _ V ||VOLUME 23_ BIRMINGHAM, ALA., SUNDAY, AUGUST 1, 1897.-SIXTEEN PAGES- NUMBER 167GREAT BRITAINIS SATISFIEDAt the Prospective Settlement ol theSeal Question.SPAIN MUST RESORT TO WARIn the Event the United States InterleresIn Cuban Altairs.HER FINANCES AT A LOW EBB,Ar.d the Struggle Can Only Bp Kept lip WhileHer Funds Last—Death Before DishonorIs the Castilian Slogan—AmericanMonopolists Denounced.London, July si.—Much satisfaction Isexpressed in official and mercantile circles at the prospective settlement of tiieseal question by aid of the Washingtonconference, especially a* such an strangement will remove ihe cause of irritationbetween the United States and GreatBritain. On the proposal of Mr. Foster';;Journey diplomats ridiculed the idea thatthere was anything necessary to ho done.Ambassador Hay and Mr. Foster havecompletely changed this idea and GreatBritain is doing everything possible tomeet the views of the United .States.Doubtless tills; is partly owing to thesupport Mr. Foster’s idea received fromRussia and Canada. Sir Wilfred Laurier and Mr. Davies have all along favored a conciliatory attitude P ward theproposals of the United States. Theconference will meet in Ihe third orfourth week in October, the exact dalebeing left to be determined by the arrivalof the British experts from the scalinggrounds. Great Britain will be repr- -Ken ted by Sir Julian Pauncefote, the ambassador. and Prof. Darcy Thompson,the United States will probably L>. represented by John W. Foster, Japan bythe Japanese minister at Washington,and Russia by a commission headed byDr. Martens, professor of internationallaw at the University of St. Petersbuig,who was delegated by Russia to hold thepour parleur with Mr. Foster. Canadawill he represented by Sir Wilfred Laurlei and Mr. Davies.The conference will discuss the wholequestion as raised by the United States;will draft a plan of protection for theseals with details of the same, and will' decide all open claims. Us report willhe ad referendum, but Russia and Japanare entirely favorable to the pie-lcn.-ionsof the United States and as Great Britainis indifferent so’ long as Canada is satisfied the conf rencB may be said to bea diplomatic triumph for the Unit dSlates and a- pel.-ona 1 triumph for Mr, :Foster. Unless it results in a dividedreport, which is not expected, the g vernments concerned will embody thestatement in a permanent agreement inthe form of a convention or me., qrandumso it can he executed in the srasua of1S99.A Spanish diplomatist who is in clo etouch with Senor Canovas d -1 Castillo,th Spanish premier. Informs the correspondent of the As- elated Press t'haa itis useless to longer disguise the fact thatthe Spanish ministry is in a very t giit jplace.He says: 'We can keep up the wa in ICuba only’ so long as our funds lust and jthe time when they will fdil is almm. insight. When it comes we must give upthe struggle. Now we are too prouu a jrace to do scat any one’s bidding, hut we ,can do so by p.eading force 'lintjuiv ina war with the United States. Such a result qptuially would lo- very u-np.cas.uitto us, but in order to ptvvciit a revolution in Spain and th'- overthrow e>i ihemottatoay wdi is the only way out.• ihe premier knows fiom 1 resident McKinley direct thait the pr -s.deiit p r. onally does not want war. and w.sr.es lo waveus from the cost and biixidwiied, but hiscabinet omy partly share his views, whilethe senate, the house of represei.'talives,■the American press and Che people of ilieUnited States favor a seu'tlemenl Willi biggluts,'President McKinley, however, while atfirst for peace, hae declare! htiueeil aw 1not shrinking from war if peace measure sfail. This is his position and naturally ilends strength to th. independent course |which Gen. Woodford win pursue. Ills 'ire iructions will be at 'ihe outset lo a.t- jtempt to secure for Cuba a peacefulmeins of Cuban lnd. pendenoe. lie is |destined to be defeated in his proposal,the moment it is made, fur Spain w midindefinitely prefer war without the en- icouragemen, of other meltons to any surren. r on the Cuban question. This .sth. facts and th' premier set s no way of .remaining Impassive in the present stale jof* Spanish polities. He will court warthe momer.it the United States becomesimperative in its d m..in.lw. Th. way ofgival emotion and p .til tlsm such as warwould create in Spain would naturallysink all threatening donr stki questions jand disturbances. The. Carl Lots, r publicans and all others would unite for thenation as they could be united by nothingel.-e and when defeated, if Spain shap'dbe defeat d, Cuba cou'd be surrenderedwithout imperilling all present politicalan! monarchial institutions.”The engineers’ dispute lias apparently jsettled down to a prolonged struggle. |Both masters and men are unyielding.Tha former’s position tins been strengthened by the fact that the Northampton ;Cycle employers havr l ined the Kmp'oy- .ers’ Federation and Issued lockout notices to iifi per cent, nf their employ es. Theengineers have responded by calling outthe other 75 per cent.. The masters helda meeting on Wednesday ard declaredthey were mere than ever determined toresist the action of the engineer*. It isalso announced, on the contrary, by trade,union statements, not signed, that a;member of the Kmployers’ Federationin ijhe ytjlhlty of London had yielded tothe'dnqiaud for an eight hour day.The Spectator, in a gloomy article, entitled 'Aggre.sl ns of American Wealth,’’referB to the retirement of President Andrews from Brown university and Prof.Bemls, from Chicago university, saying:Wa cun scarcely escape Ihe coiiclu- 1slon that a conspiracy exists among thevarious trusts In America to interferewith the freedom of teaching In theuniversities, so far as economic questions are concerned. in order that nothingshall be uttered which has not receivedthe trade mark of monopolist approval.'The paper pictures the Americans asreduced almost to the situation of Russia by the censorship of wealthy magnates ov-r the universities and says:'Capitalists have subsidized the pulpits. bought up the press, seated wellpaid attorneys in the senate and finallyhave Stretched th tr hands out to thecolleges which it is an easy thing to capture by such generosity as Mr. Rockefeller's. Apnnrt-ntly it i» their intentionto culvert til? Unit' il States ittin a powerful oligarchy and they will i xtend thesway rf that oligarchy to other land*when they can.'Stephen Crane has announced that helikes England so well that h ■ will make' London his home. Mr. Crane's bookshave found their greatest vogue in thiscountry and he is receiving considerable attention in literary clrcl s here.DOINGSOF AN INSANE MAN,'Interests Himself In Flanagan and EmploysCcunstI to Defend an Imaginary CriminalAtlanta, Ga„ July i!1.—J. L. Moyse,repro nting hlrne-lf to be of the Arm ofLloyd. Moyse & West, attorn ys, of NewYork, appeared here Monday and sp.-ntthe morning as a spectator of the famousFlanagan, murder case, in which the d -fen.se was insanity. In the afternoonMoyne approached Col. W. C. GI nil,leading counsel for Fin agon, and introduced him*, if. saying h had come to employ Glenr.v to aid him in the defense of anotable murder case at Home, Ga. Hesaid he had then sent to Georgia by theCrescent lire wing company to defend H.L. Matthews, having been an. agent of thebrewing company, the company had putup unlimited money to defray the exporises of the defense. He went into thedetails of the crime and the law in thecase in a very intelligent manner amiended by employing Col. Glenn for a feeof SI,000 to aid in defending the pr soner.Sine - that time be has been a spectatorat the Flanagan trial, watching with absorbed Interest the highly irttepesting deft.,ns-j of Flanagan, which was based uponthe defendant’s affliction with paranoia.On the strength of -his h-avi g engag'dCol. Glenn in the Mathews case Moyseassisted in the-defers.' of Flanagan andmade a most valuable aid, showing himBelf to be an expert in ail matter pertaining to mental derangements. The caseof Flanagan was concluded today andthis afternoon a conference was held atCol. Glenn"
print(len(xxx))